In [1]:
import requests
import datetime
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

In [3]:
api = "https://api.thegraph.com/subgraphs/name/convex-community/tricrypto"

In [4]:
query = """
{
  exchangeEvents(first: 1000) {
    id
    timestamp
    blockNumber
    txHash
    address
    amountUSDSold
    amountUSDBought
    amountETHSold
    amountETHSoldUSD
    amountETHBought
    amountETHBoughtUSD
    amountBTCSold
    amountBTCSoldUSD
    amountBTCBought
    amountBTCBoughtUSD
    totalSoldUSD
    totalBoughtUSD
    traderFeesUSD
    assetPrices {
      blockNumber
      timestamp
      btcPrice
      ethPrice
    }
    poolSnapshot {
      ethBalance
      ethBalanceUSD
      btcBalance
      btcBalanceUSD
      usdtBalance
      ethScalePrice
      btcScalePrice
      feeFraction
    }
  }
}
"""

r = requests.post(api, json={'query': query})
data = dict(r.json())
data

{'data': {'exchangeEvents': [{'address': '0xcd9e13b2f3bfc26cd99989fb849a2751932595c4',
    'amountBTCBought': '0',
    'amountBTCBoughtUSD': '0',
    'amountBTCSold': '3.79796632',
    'amountBTCSoldUSD': '176205.83552574403330666922089984',
    'amountETHBought': '0',
    'amountETHBoughtUSD': '0',
    'amountETHSold': '0',
    'amountETHSoldUSD': '0',
    'amountUSDBought': '176191.519056',
    'amountUSDSold': '0',
    'assetPrices': {'blockNumber': '13190530',
     'btcPrice': '46394.786230159100859712',
     'ethPrice': '3542.832523234121946961',
     'timestamp': '1631177128'},
    'blockNumber': '13190530',
    'id': '0x0000a051465f674c738951bbee1363c43501ec29a8bdcea958cf7749fd8c8e3c-217',
    'poolSnapshot': {'btcBalance': '3606.11156976',
     'btcBalanceUSD': '167304775.4011186678557662058615091',
     'btcOraclePrice': '46394.786230159100859712',
     'btcScalePrice': '46210.070617015880804731',
     'ethBalance': '46982.421726479249592682',
     'ethBalanceUSD': '166450851.

In [9]:
swaps = data['data']['exchangeEvents']
de_nested_swap_txes = []
for swap in swaps:

    pool_snapshot = swap['poolSnapshot']
    asset_prices = swap['assetPrices']
    swap_timestamp = int(swap['timestamp'])
    swap_time = datetime.datetime.fromtimestamp(swap_timestamp)
    tvl_usd = float(pool_snapshot['usdtBalance']) + float(pool_snapshot['ethBalanceUSD']) + float(pool_snapshot['btcBalanceUSD'])

    de_nested_swap_txes.append(
        {
            'swap_time': swap_time,
            'block_number': int(swap['blockNumber']),
            'tx_hash': swap['txHash'],
            'address': swap['address'],
            'amount_btc_bought': float(swap['amountBTCBought']),
            'amount_btc_bought_usd': float(swap['amountBTCBoughtUSD']),
            'amount_btc_sold': float(swap['amountBTCSold']),
            'amount_btc_sold_usd': float(swap['amountBTCSoldUSD']),
            'amount_eth_bought': float(swap['amountETHBought']),
            'amount_eth_bought_usd': float(swap['amountETHBoughtUSD']),
            'amount_eth_sold': float(swap['amountETHSold']),
            'amount_eth_sold_usd': float(swap['amountETHSoldUSD']),
            'amount_usdt_bought': float(swap['amountUSDBought']),
            'amount_usdt_sold': float(swap['amountUSDSold']),
            'swap_fee_usd': float(swap['traderFeesUSD']),
            'fee_fraction': float(pool_snapshot['feeFraction']),
            'btc_price_usd': float(asset_prices['btcPrice']),
            'btc_scale_price_usd': float(pool_snapshot['btcScalePrice']),
            'eth_price_usd': float(asset_prices['ethPrice']),
            'eth_scale_price_usd': float(pool_snapshot['ethScalePrice']),
            'btc_balance': float(pool_snapshot['btcBalance']),
            'btc_balance_usd': float(pool_snapshot['btcBalanceUSD']),
            'eth_balance': float(pool_snapshot['ethBalance']),
            'eth_balance_usd': float(pool_snapshot['ethBalanceUSD']),
            'usdt_balance': float(pool_snapshot['usdtBalance']),
            'tvl_usd': tvl_usd
        }
    )

df_swaps = pd.DataFrame(de_nested_swap_txes)
df_swaps

,swap_time,block_number,tx_hash,address,amount_btc_bought,amount_btc_bought_usd,amount_btc_sold,amount_btc_sold_usd,amount_eth_bought,amount_eth_bought_usd,...,btc_price_usd,btc_scale_price_usd,eth_price_usd,eth_scale_price_usd,btc_balance,btc_balance_usd,eth_balance,eth_balance_usd,usdt_balance,tvl_usd
0,2021-09-09 10:45:28,13190530,0x0000a051465f674c738951bbee1363c43501ec29a8bd...,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,0.000000,0.000000,3.797966,176205.835526,0.000000,0.000000,...,46394.786230,46210.070617,3542.832523,3505.639408,3606.111570,1.673048e+08,46982.421726,1.664509e+08,1.679488e+08,5.017044e+08
1,2021-10-21 19:36:39,13462408,0x0001bf5585a38f102397bd2b8cba837b15ced82c8a3b...,0xfa9a30350048b2bf66865ee20363067c66f67e58,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,63176.257882,64558.258992,4095.446879,4179.113033,4151.349994,2.622668e+08,64063.815289,2.623700e+08,2.607801e+08,7.854168e+08
2,2021-11-16 22:03:42,13628866,0x00032be1982bf81ffc3c56d3c878c310c42995d50552...,0xbf3f6477dbd514ef85b7d3ec6ac2205fd0962039,4.684653,279318.195992,0.000000,0.000000,0.000000,0.000000,...,59624.090011,60222.561027,4189.913089,4259.703766,5647.829094,3.367467e+08,80662.783003,3.379701e+08,3.362105e+08,1.010927e+09
3,2021-11-16 06:39:42,13624827,0x000342cf18da059253b2acd74add8e783d4e41ef4560...,0x4d246be90c2f36730bb853ad41d0a189061192d3,0.000000,0.000000,4.904030,300595.706945,0.000000,0.000000,...,61295.650097,61630.553215,4349.439004,4358.515731,5636.301014,3.454807e+08,78976.844826,3.435050e+08,3.426419e+08,1.031628e+09
4,2021-10-13 14:39:14,13410034,0x0007e54f83ee62c35a6d7c3a813c82b5b166ba858d52...,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,55169.614351,55766.736624,3450.218491,3486.726861,4473.450237,2.467985e+08,71058.742675,2.451682e+08,2.449146e+08,7.368813e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021-10-01 09:27:38,13332090,0x07761a80290d9473bda341431866faa01ebb0cc0d3d2...,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,2.682304,118049.887005,0.000000,0.000000,0.000000,0.000000,...,44010.624046,44004.583735,3033.276243,3032.297860,4308.815537,1.896337e+08,62550.476818,1.897329e+08,1.929832e+08,5.723498e+08
996,2021-09-19 01:58:51,13252806,0x0777e81e50ac6765e3dadcf8288096406b32a8af0612...,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,0.000000,0.000000,0.000000,0.000000,115.143307,393745.186288,...,48162.779270,48143.951740,3419.609835,3428.095946,3929.572512,1.892591e+08,55549.492736,1.899576e+08,1.903959e+08,5.696126e+08
997,2021-09-04 14:00:41,13159074,0x07785dbd09bf36d7a54a1cb8500646aa80c59d9371a1...,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,50058.631524,49901.735785,3904.196357,3865.586347,3467.936776,1.736002e+08,44004.698469,1.718030e+08,1.734202e+08,5.188234e+08
998,2021-11-23 08:20:34,13669479,0x077a056a34fb0ba8e7598b6bb600c33204f8d49ba9bf...,0x5bd9dee999fe2c4e6efdbab2c395b4685b0c00d0,0.000000,0.000000,3.400000,191567.865126,0.000000,0.000000,...,56343.489743,56377.372328,4131.270738,4107.191914,5825.638152,3.282368e+08,79024.602822,3.264720e+08,3.255471e+08,9.802559e+08


In [10]:
df_swaps.set_index('swap_time', inplace=True)
df_swaps

,block_number,tx_hash,address,amount_btc_bought,amount_btc_bought_usd,amount_btc_sold,amount_btc_sold_usd,amount_eth_bought,amount_eth_bought_usd,amount_eth_sold,...,btc_price_usd,btc_scale_price_usd,eth_price_usd,eth_scale_price_usd,btc_balance,btc_balance_usd,eth_balance,eth_balance_usd,usdt_balance,tvl_usd
swap_time,,,,,,,,,,,,,,,,,,,,,
2021-09-09 10:45:28,13190530,0x0000a051465f674c738951bbee1363c43501ec29a8bd...,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,0.000000,0.000000,3.797966,176205.835526,0.000000,0.000000,0.000000,...,46394.786230,46210.070617,3542.832523,3505.639408,3606.111570,1.673048e+08,46982.421726,1.664509e+08,1.679488e+08,5.017044e+08
2021-10-21 19:36:39,13462408,0x0001bf5585a38f102397bd2b8cba837b15ced82c8a3b...,0xfa9a30350048b2bf66865ee20363067c66f67e58,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,...,63176.257882,64558.258992,4095.446879,4179.113033,4151.349994,2.622668e+08,64063.815289,2.623700e+08,2.607801e+08,7.854168e+08
2021-11-16 22:03:42,13628866,0x00032be1982bf81ffc3c56d3c878c310c42995d50552...,0xbf3f6477dbd514ef85b7d3ec6ac2205fd0962039,4.684653,279318.195992,0.000000,0.000000,0.000000,0.000000,0.000000,...,59624.090011,60222.561027,4189.913089,4259.703766,5647.829094,3.367467e+08,80662.783003,3.379701e+08,3.362105e+08,1.010927e+09
2021-11-16 06:39:42,13624827,0x000342cf18da059253b2acd74add8e783d4e41ef4560...,0x4d246be90c2f36730bb853ad41d0a189061192d3,0.000000,0.000000,4.904030,300595.706945,0.000000,0.000000,0.000000,...,61295.650097,61630.553215,4349.439004,4358.515731,5636.301014,3.454807e+08,78976.844826,3.435050e+08,3.426419e+08,1.031628e+09
2021-10-13 14:39:14,13410034,0x0007e54f83ee62c35a6d7c3a813c82b5b166ba858d52...,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.000000,...,55169.614351,55766.736624,3450.218491,3486.726861,4473.450237,2.467985e+08,71058.742675,2.451682e+08,2.449146e+08,7.368813e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01 09:27:38,13332090,0x07761a80290d9473bda341431866faa01ebb0cc0d3d2...,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,2.682304,118049.887005,0.000000,0.000000,0.000000,0.000000,0.000000,...,44010.624046,44004.583735,3033.276243,3032.297860,4308.815537,1.896337e+08,62550.476818,1.897329e+08,1.929832e+08,5.723498e+08
2021-09-19 01:58:51,13252806,0x0777e81e50ac6765e3dadcf8288096406b32a8af0612...,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,0.000000,0.000000,0.000000,0.000000,115.143307,393745.186288,0.000000,...,48162.779270,48143.951740,3419.609835,3428.095946,3929.572512,1.892591e+08,55549.492736,1.899576e+08,1.903959e+08,5.696126e+08
2021-09-04 14:00:41,13159074,0x07785dbd09bf36d7a54a1cb8500646aa80c59d9371a1...,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,61.278612,...,50058.631524,49901.735785,3904.196357,3865.586347,3467.936776,1.736002e+08,44004.698469,1.718030e+08,1.734202e+08,5.188234e+08
